# Tutorial Simplex
This is based on this [link.](https://www.softcover.io/read/7b8eb7d0/juliabook/simplex)

In [48]:
using Pkg
Pkg.add("Clp")
Pkg.add("JuMP")
Pkg.add("Cbc");
Pkg.add("MathProgBase");
Pkg.add("Combinatorics");
using StatsPlots, Plots
using Combinatorics, LinearAlgebra, Printf

 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


### **1.** Searching all Basic Feasible Solutions

In [49]:
function isnonnegative(x::Array{Float64, 1})
  return length( x[ x .< 0] ) == 0
end

function searchBFS(c, A, b)
  m, n = size(A)
  @assert rank(A) == m

  opt_x = zeros(n)
  obj = Inf

  for b_idx in combinations(1:n, m)
    B = A[:, b_idx]
    c_B = c[b_idx]
    x_B = inv(B) * b

    if isnonnegative(x_B)
      z = dot(c_B, x_B)
      if z < obj
        obj = z
        opt_x = zeros(n)
        opt_x[b_idx] = x_B
      end
    end

    println("Basis:", b_idx)
    println("\t x_B = ", x_B)
    println("\t nonnegative? ", isnonnegative(x_B))
    if isnonnegative(x_B)
      println("\t obj = ", dot(c_B, x_B))
    end

  end

  return opt_x, obj
end

searchBFS (generic function with 1 method)

In [50]:
c = [-3; -2; -1; -5; 0; 0; 0]
A = [7 3 4 1 1 0 0 ;
     2 1 1 5 0 1 0 ;
     1 4 5 2 0 0 1 ]
b = [7; 3; 8]
opt_x, obj = searchBFS(c, A, b)

Basis:[1, 2, 3]
	 x_B = [0.25, 4.75, -2.25]
	 nonnegative? false
Basis:[1, 2, 4]
	 x_B = [0.172414, 1.87931, 0.155172]
	 nonnegative? true
	 obj = -5.051724137931034
Basis:[1, 2, 5]
	 x_B = [0.571429, 1.85714, -2.57143]
	 nonnegative? false
Basis:[1, 2, 6]
	 x_B = [0.16, 1.96, 0.72]
	 nonnegative? true
	 obj = -4.3999999999999995
Basis:[1, 2, 7]
	 x_B = [-2.0, 7.0, -18.0]
	 nonnegative? false
Basis:[1, 3, 4]
	 x_B = [0.121622, 1.47297, 0.256757]
	 nonnegative? true
	 obj = -3.121621621621621
Basis:[1, 3, 5]
	 x_B = [0.777778, 1.44444, -4.22222]
	 nonnegative? false
Basis:[1, 3, 6]
	 x_B = [0.0967742, 1.58065, 1.22581]
	 nonnegative? true
	 obj = -1.8709677419354835
Basis:[1, 3, 7]
	 x_B = [5.0, -7.0, 38.0]
	 nonnegative? false
Basis:[1, 4, 5]
	 x_B = [34.0, -13.0, -218.0]
	 nonnegative? false
Basis:[1, 4, 6]
	 x_B = [0.461538, 3.76923, -16.7692]
	 nonnegative? false
Basis:[1, 4, 7]
	 x_B = [0.969697, 0.212121, 6.60606]
	 nonnegative? true
	 obj = -3.9696969696969693
Basis:[1, 5, 6]
	 x

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931034)

### **2.** Using solvers to solve the LP problem

In [51]:
using JuMP, Clp, MathProgBase

In [52]:
Af = convert(Array{Float64}, A)
cf = convert(Array{Float64}, c)
bf = convert(Array{Float64}, b)

sol = linprog(cf, Af, '=', bf, ClpSolver()) #Only works with Float instead of Int
sol.status, sol.objval, sol.sol

(:Optimal, -5.051724137931035, [0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0])

### **3.** Implementing the Simplex
Note that this code is different from the original due to the use of a more recent Julia version.

In [334]:
  mutable struct simplex_tableau
    z_c     ::Array{Float64}
    Y       ::Array{Float64}
    x_B     ::Array{Float64}
    obj     ::Float64
    b_idx   ::Array{Int64}
  end

  function isnonnegative(x::Array{Float64})
    return length( x[ x .< 0] ) == 0
  end

  function initial_BFS(A, b)
    m, n = size(A)

    comb = collect(combinations(1:n, m))
    for i in length(comb):-1:1
      b_idx = comb[i]
      B = A[:, b_idx]
      x_B = inv(B) * b
      if isnonnegative(x_B)
        return b_idx, x_B, B
      end
    end

    error("Infeasible")
  end

  function print_tableau(t::simplex_tableau)
    m, n = size(t.Y)

    hline0 = repeat("-", 6)
    hline1 = repeat("-", 7*n)
    hline2 = repeat("-", 7)
    hline = join([hline0, "+", hline1, "+", hline2])

    println(hline)

    @printf("%6s|", "")
    for j in 1:length(t.z_c)
      @printf("%6.2f ", t.z_c[j])
    end
    @printf("| %6.2f\n", t.obj)

    println(hline)

    for i in 1:m
      @printf("x[%2d] |", t.b_idx[i])
      for j in 1:n
        @printf("%6.2f ", t.Y[i,j])
      end
      @printf("| %6.2f\n", t.x_B[i])
    end

    println(hline)
  end

  function pivoting!(t::simplex_tableau)
    m, n = size(t.Y)

    entering, exiting = pivot_point(t)
    println("Pivoting: entering = x_$entering, exiting = x_$(t.b_idx[exiting])")

    # Pivoting: exiting-row, entering-column
    # updating exiting-row
    coef = t.Y[exiting, entering]
    t.Y[exiting, :] /= coef
    t.x_B[exiting] /= coef

    # updating other rows of Y
    for i in setdiff(1:m, exiting)
      coef = t.Y[i, entering]
      t.Y[i, :] -= coef * t.Y[exiting, :]
      t.x_B[i] -= coef * t.x_B[exiting]
    end

    # updating the row for the reduced costs
    coef = t.z_c[entering]
    t.z_c .-= coef * t.Y[exiting, :]'
    t.obj -= coef * t.x_B[exiting]

    # Updating b_idx
    t.b_idx[ findall(t.b_idx.==t.b_idx[exiting])[1] ] = entering
  end

  function pivot_point(t::simplex_tableau)
    # Finding the entering variable index
    entering_aux = findfirst(t.z_c .> 0) # Auxiliary variable
    if entering_aux == nothing
      error("Optimal")
    end
    entering = entering_aux[2]

    # min ratio test / finding the exiting variable index
    pos_idx = findall( t.Y[:, entering] .> 0 )
    if length(pos_idx) == 0
      error("Unbounded")
    end
    exiting = pos_idx[ argmin( t.x_B[pos_idx] ./ t.Y[pos_idx, entering] ) ]

    return entering, exiting
  end

  function initialize(c, A, b)
    c = Array{Float64}(c)
    A = Array{Float64}(A)
    b = Array{Float64}(b)

    m, n = size(A)

    # Finding an initial BFS
    b_idx, x_B, B = initial_BFS(A,b)

    Y = inv(B) * A
    c_B = c[b_idx]
    obj = dot(c_B, x_B)

    # z_c is a row vector
    z_c = zeros(1,n)
    n_idx = setdiff(1:n, b_idx)
    z_c[n_idx] = c_B' * inv(B) * A[:,n_idx] - c[n_idx]'

    return simplex_tableau(z_c, Y, x_B, obj, b_idx)
  end

  function isOptimal(tableau)
    return findfirst( tableau.z_c .> 0 ) == nothing
  end

  function simplex_method(c, A, b)
    tableau = initialize(c, A, b)
    print_tableau(tableau)

    while !isOptimal(tableau)
      pivoting!(tableau)
      print_tableau(tableau)
    end

    opt_x = zeros(length(c))
    opt_x[tableau.b_idx] = tableau.x_B

    return opt_x, tableau.obj
  end

simplex_method (generic function with 1 method)

In [335]:
c = [-3; -2; -1; -5; 0; 0; 0]
A = [7 3 4 1 1 0 0 ;
     2 1 1 5 0 1 0 ;
     1 4 5 2 0 0 1 ]
b = [7; 3; 8]
# include("simplex_method.jl")
# using SimplexMethod

Results = simplex_method(c, A, b)

------+-------------------------------------------------+-------
      |  3.00   2.00   1.00   5.00   0.00   0.00   0.00 |   0.00
------+-------------------------------------------------+-------
x[ 5] |  7.00   3.00   4.00   1.00   1.00   0.00   0.00 |   7.00
x[ 6] |  2.00   1.00   1.00   5.00   0.00   1.00   0.00 |   3.00
x[ 7] |  1.00   4.00   5.00   2.00   0.00   0.00   1.00 |   8.00
------+-------------------------------------------------+-------
Pivoting: entering = x_1, exiting = x_5
------+-------------------------------------------------+-------
      |  0.00   0.71  -0.71   4.57  -0.43   0.00   0.00 |  -3.00
------+-------------------------------------------------+-------
x[ 1] |  1.00   0.43   0.57   0.14   0.14   0.00   0.00 |   1.00
x[ 6] |  0.00   0.14  -0.14   4.71  -0.29   1.00   0.00 |   1.00
x[ 7] |  0.00   3.57   4.43   1.86  -0.14   0.00   1.00 |   7.00
------+-------------------------------------------------+-------
Pivoting: entering = x_2, exiting = x_7
------+---

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931035)

In [336]:
Results

([0.172414, 1.87931, 0.0, 0.155172, 0.0, 0.0, 0.0], -5.051724137931035)